In [43]:
import pandas as pd


df_clean_2015 = pd.read_csv('../data/clean/2015_clean.csv')
df_clean_2016 = pd.read_csv('../data/clean/2016_clean.csv')  
df_clean_2017 = pd.read_csv('../data/clean/2017_clean.csv')  
df_clean_2018 = pd.read_csv('../data/clean/2018_clean.csv')  
df_clean_2019 = pd.read_csv('../data/clean/2019_clean.csv')  


In [44]:
# Crear un DataFrame auxiliar con 'Country' y 'Region' de 2015 y 2016, sin duplicados
df_region = pd.concat([df_clean_2015[['Country', 'Region']], df_clean_2016[['Country', 'Region']]]).drop_duplicates()

# Función para agregar la columna 'Region' después de 'Country'
def merge_region_column(df, df_region):
    df = df.merge(df_region, on='Country', how='left')
    # Reordenar la columna 'Region' para que aparezca justo después de 'Country'
    cols = df.columns.tolist()
    idx = cols.index('Country') + 1
    cols.insert(idx, cols.pop(cols.index('Region')))
    df = df[cols]
    return df

# Aplicar la función a cada dataset que necesita la columna 'Region'
df_clean_2017 = merge_region_column(df_clean_2017, df_region)
df_clean_2018 = merge_region_column(df_clean_2018, df_region)
df_clean_2019 = merge_region_column(df_clean_2019, df_region)

# Comprobación: mostrar el conteo y los valores nulos de cada dataset después del merge
for year, df in zip([2017, 2018, 2019], [df_clean_2017, df_clean_2018, df_clean_2019]):
    print(f"\nValores nulos en la columna 'Region' para el año {year}:")
    print(df['Region'].isnull().sum())
    print(df[df['Region'].isnull()][['Country', 'Region']])




Valores nulos en la columna 'Region' para el año 2017:
2
                     Country Region
32  Taiwan Province of China    NaN
70   Hong Kong S.A.R., China    NaN

Valores nulos en la columna 'Region' para el año 2018:
2
              Country Region
37  Trinidad & Tobago    NaN
57    Northern Cyprus    NaN

Valores nulos en la columna 'Region' para el año 2019:
4
               Country Region
38   Trinidad & Tobago    NaN
63     Northern Cyprus    NaN
83     North Macedonia    NaN
119             Gambia    NaN


In [45]:
# Verificar si los países con valores nulos tienen nombres alternativos en el dataset de 2015
missing_regions = df_clean_2017[df_clean_2017['Region'].isnull()]['Country']
print("\nPaíses con valores nulos en 'Region':\n", missing_regions)

print("\nPosibles coincidencias en el dataset de 2015:")
for country in missing_regions:
    similar_names = df_clean_2015[df_clean_2015['Country'].str.contains(country.split()[0], case=False, na=False)]
    print(f"{country} posibles coincidencias en 2015:\n", similar_names[['Country', 'Region']], "\n")

# Configurar pandas para mostrar más filas y columnas si es necesario
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)


Países con valores nulos en 'Region':
 32    Taiwan Province of China
70     Hong Kong S.A.R., China
Name: Country, dtype: object

Posibles coincidencias en el dataset de 2015:
Taiwan Province of China posibles coincidencias en 2015:
    Country        Region
37  Taiwan  Eastern Asia 

Hong Kong S.A.R., China posibles coincidencias en 2015:
       Country        Region
71  Hong Kong  Eastern Asia 



Este siguiente codigo es fallido, ya que solo estoy usando la primer palabra como coincidencia y no debe ser asi, esto limita la búsqueda para nombres con términos compuestos, como “Northern Cyprus.

In [46]:
# Verificar si los países con valores nulos tienen nombres alternativos en el dataset de 2015
missing_regions = df_clean_2018[df_clean_2018['Region'].isnull()]['Country']
print("\nPaíses con valores nulos en 'Region':\n", missing_regions)

print("\nPosibles coincidencias en el dataset de 2015:")
for country in missing_regions:
    similar_names = df_clean_2015[df_clean_2015['Country'].str.contains(country.split()[0], case=False, na=False)]
    print(f"{country} posibles coincidencias en 2015:\n", similar_names[['Country', 'Region']], "\n")

# Configurar pandas para mostrar más filas y columnas si es necesario
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)


Países con valores nulos en 'Region':
 37    Trinidad & Tobago
57      Northern Cyprus
Name: Country, dtype: object

Posibles coincidencias en el dataset de 2015:
Trinidad & Tobago posibles coincidencias en 2015:
                 Country                       Region
40  Trinidad and Tobago  Latin America and Caribbean 

Northern Cyprus posibles coincidencias en 2015:
 Empty DataFrame
Columns: [Country, Region]
Index: [] 



Para estos casos utilizamos "get_close_matches" para encontrar coincidencias en los nombres basándose en un umbral (cutoff=0.6), esto lo podemos ajustar en el codigo segun nuestro deseo

In [47]:
from difflib import get_close_matches

# Función para encontrar coincidencias cercanas
def find_similar_country(country_name, df_reference, threshold=0.6):
    similar_countries = get_close_matches(country_name, df_reference['Country'], n=1, cutoff=threshold)
    if similar_countries:
        return df_reference[df_reference['Country'] == similar_countries[0]][['Country', 'Region']]
    else:
        return pd.DataFrame(columns=['Country', 'Region'])

# Ejemplo de aplicación al dataset de 2018
missing_regions = df_clean_2018[df_clean_2018['Region'].isnull()]['Country']
print("\nPaíses con valores nulos en 'Region':\n", missing_regions)

print("\nPosibles coincidencias en el dataset de 2015:")
for country in missing_regions:
    similar_names = find_similar_country(country, df_clean_2015)
    print(f"{country} posibles coincidencias en 2015:\n", similar_names, "\n")



Países con valores nulos en 'Region':
 37    Trinidad & Tobago
57      Northern Cyprus
Name: Country, dtype: object

Posibles coincidencias en el dataset de 2015:
Trinidad & Tobago posibles coincidencias en 2015:
                 Country                       Region
40  Trinidad and Tobago  Latin America and Caribbean 

Northern Cyprus posibles coincidencias en 2015:
          Country          Region
65  North Cyprus  Western Europe 



In [49]:
# Función para encontrar coincidencias cercanas
def find_similar_country(country_name, df_reference, threshold=0.7):
    similar_countries = get_close_matches(country_name, df_reference['Country'], n=1, cutoff=threshold)
    if similar_countries:
        return df_reference[df_reference['Country'] == similar_countries[0]][['Country', 'Region']]
    else:
        return pd.DataFrame(columns=['Country', 'Region'])

# Ejemplo de aplicación al dataset de 2019
missing_regions = df_clean_2019[df_clean_2019['Region'].isnull()]['Country']
print("\nPaíses con valores nulos en 'Region':\n", missing_regions)

print("\nPosibles coincidencias en el dataset de 2015:")
for country in missing_regions:
    similar_names = find_similar_country(country, df_clean_2015)
    print(f"{country} posibles coincidencias en 2015:\n", similar_names, "\n")



Países con valores nulos en 'Region':
 38     Trinidad & Tobago
63       Northern Cyprus
83       North Macedonia
119               Gambia
Name: Country, dtype: object

Posibles coincidencias en el dataset de 2015:
Trinidad & Tobago posibles coincidencias en 2015:
                 Country                       Region
40  Trinidad and Tobago  Latin America and Caribbean 

Northern Cyprus posibles coincidencias en 2015:
          Country          Region
65  North Cyprus  Western Europe 

North Macedonia posibles coincidencias en 2015:
       Country                      Region
92  Macedonia  Central and Eastern Europe 

Gambia posibles coincidencias en 2015:
    Country              Region
84  Zambia  Sub-Saharan Africa 



In [52]:
# Diccionario para mapear nombres alternativos a una versión estándar
country_corrections = {
    'Taiwan Province of China': 'Taiwan',
    'Hong Kong S.A.R., China': 'Hong Kong',
    'Trinidad & Tobago': 'Trinidad and Tobago',
    'Northern Cyprus': 'North Cyprus',
    'North Macedonia': 'Macedonia',
    'Gambia': 'Gambia'  # Se añadirá región de forma manual
}

In [53]:
# Función para encontrar coincidencias de regiones en el dataset de 2015 y asignarlas
def assign_missing_regions(df_target, df_reference):
    for idx, row in df_target[df_target['Region'].isnull()].iterrows():
        country_name = row['Country']
        
        # Buscar coincidencia en el dataset de 2015 usando la función de similitud
        similar_names = find_similar_country(country_name, df_reference)
        
        # Si se encuentra una coincidencia, asignar la región
        if not similar_names.empty:
            df_target.loc[idx, 'Region'] = similar_names.iloc[0]['Region']
        else:
            print(f"No se encontró coincidencia para: {country_name}")

# Aplicar la función a los datasets de 2017, 2018 y 2019 usando el dataset de 2015 como referencia
for df in [df_clean_2017, df_clean_2018, df_clean_2019]:
    assign_missing_regions(df, df_clean_2015)

# Asignar manualmente la región para "Gambia"
for df in [df_clean_2017, df_clean_2018, df_clean_2019]:
    df.loc[df['Country'] == 'Gambia', 'Region'] = 'Sub-Saharan Africa'

# Estandarizar nombres de países una vez completadas las regiones
for df in [df_clean_2017, df_clean_2018, df_clean_2019]:
    df['Country'] = df['Country'].replace(country_corrections)

# Verificar que no queden valores nulos en 'Region'
for year, df in zip([2017, 2018, 2019], [df_clean_2017, df_clean_2018, df_clean_2019]):
    print(f"Valores nulos en 'Region' para el año {year} después de la corrección:")
    print(df['Region'].isnull().sum())


Valores nulos en 'Region' para el año 2017 después de la corrección:
0
Valores nulos en 'Region' para el año 2018 después de la corrección:
0
Valores nulos en 'Region' para el año 2019 después de la corrección:
0


In [56]:
# Guardar los DataFrames en la carpeta data/clean
df_clean_2017.to_csv('../data/clean/2017_clean_with_region.csv', index=False)
df_clean_2018.to_csv('../data/clean/2018_clean_with_region.csv', index=False)
df_clean_2019.to_csv('../data/clean/2019_clean_with_region.csv', index=False)

# Confirmación de guardado
print("Archivos guardados exitosamente en la carpeta 'data/clean'")


Archivos guardados exitosamente en la carpeta 'data/clean'


In [6]:
import pandas as pd

# Cargar los datasets limpios de cada año
df_clean_2015 = pd.read_csv('../data/clean/2015_clean.csv')
df_clean_2016 = pd.read_csv('../data/clean/2016_clean.csv') 
df_clean_2017 = pd.read_csv('../data/clean/2017_clean_with_region.csv')
df_clean_2018 = pd.read_csv('../data/clean/2018_clean_with_region.csv')
df_clean_2019 = pd.read_csv('../data/clean/2019_clean_with_region.csv')

# Concatenar los datasets a lo largo de las filas
df_final = pd.concat([df_clean_2015, df_clean_2016, df_clean_2017, df_clean_2018, df_clean_2019], ignore_index=True)

# Guardar el dataset final combinado
df_final.to_csv('../data/proccesed/final_happiness_data.csv', index=False)
print("El dataset combinado ha sido guardado en 'data/clean/final_happiness_data.csv'")


El dataset combinado ha sido guardado en 'data/clean/final_happiness_data.csv'


In [ ]:
# Configurar pandas para mostrar más filas y columnas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Mostrar las primeras filas para exploración inicial
df_final.head(160)

In [8]:
# Verificar el número de filas y columnas del dataset final
num_filas, num_columnas = df_final.shape
print(f"El dataset final tiene {num_filas} filas y {num_columnas} columnas.")


El dataset final tiene 782 filas y 9 columnas.
